In [1]:
import tensorflow_model_analysis as tfma
import tensorflow as tf
import sys
import os
import logging


In [2]:
eval_shared_model=tfma.default_eval_shared_model(
    eval_saved_model_path="/Users/bhuwankarki/Desktop/ops/building_pipelines/ml_pipeline/Trainer/model/36/Format-Serving",
    tags=[tf.saved_model.SERVING]
)

In [15]:
eval_shared_model

EvalSharedModel(model_path='/Users/bhuwankarki/Desktop/ops/building_pipelines/ml_pipeline/Trainer/model/36/Format-Serving', add_metrics_callbacks=[], include_default_metrics=True, example_weight_key=None, additional_fetches=None, model_loader=<tensorflow_model_analysis.types.ModelLoader object at 0x7fda89b2b6c0>, model_name='', model_type='tf_estimator', rubber_stamp=False, is_baseline=False)

In [3]:
slice=[tfma.slicer.SingleSliceSpec(),
       tfma.slicer.SingleSliceSpec(columns=['product'])]

In [17]:
eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name='BinaryAccuracy'),
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='FalsePositives'),
                tfma.MetricConfig(class_name='TruePositives'),
                tfma.MetricConfig(class_name='FalseNegatives'),
                tfma.MetricConfig(class_name='TrueNegatives')
              ])])


In [18]:
eval_result=tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                                    eval_config=eval_config,
                                    data_location="/Users/bhuwankarki/Desktop/ops/building_pipelines/ml_pipeline/CsvExampleGen/examples/17/Split-eval/data_tfrecord-00000-of-00001.gz",
                                    output_path="./eval_result_2000_steps",
                                    file_format='tfrecords',
                                    slice_spec=slice)

In [19]:
tfma.view.render_slicing_metrics(eval_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [20]:
tfma.view.render_slicing_metrics(eval_result, slicing_spec=slice[1])

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'product:Debt collect…

In [12]:
!jupyter nbextension enable --py widgetsnbextension 
!jupyter nbextension install --py \
                       --symlink tensorflow_model_analysis
!jupyter nbextension enable --py tensorflow_model_analysis

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Installing /opt/anaconda3/lib/python3.8/site-packages/tensorflow_model_analysis/static -> tensorflow_model_analysis
Symlinking: /usr/local/share/jupyter/nbextensions/tensorflow_model_analysis -> /opt/anaconda3/lib/python3.8/site-packages/tensorflow_model_analysis/static
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable tensorflow_model_analysis --py
    
Enabling notebook extension tensorflow_model_analysis/extension...
      - Validating: OK


In [14]:
!pip install tensorboard_plugin_fairness_indicators

In [4]:
eval_config_fairness=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                  tfma.MetricConfig(class_name='BinaryAccuracy'),
                  tfma.MetricConfig(class_name='ExampleCount'),
                  tfma.MetricConfig(class_name='FalsePositives'),
                  tfma.MetricConfig(class_name='TruePositives'),
                  tfma.MetricConfig(class_name='FalseNegatives'),
                  tfma.MetricConfig(class_name='TrueNegatives'),
                  tfma.MetricConfig(class_name='FairnessIndicators', config='{"thresholds":[0.25, 0.5, 0.75]}')
              ])])

In [6]:
eval_result=tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                                    eval_config=eval_config_fairness,
                                    data_location="/Users/bhuwankarki/Desktop/ops/building_pipelines/ml_pipeline/CsvExampleGen/examples/17/Split-eval/data_tfrecord-00000-of-00001.gz",
                                    output_path="./eval_result_fairness",
                                    file_format='tfrecords',
                                    slice_spec=slice)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [7]:

from tensorboard_plugin_fairness_indicators import summary_v2

In [8]:
writer = tf.summary.create_file_writer('./fairness_indicator_logs')
with writer.as_default():
    summary_v2.FairnessIndicators('./eval_result_fairness', step=1)
writer.close()

In [11]:
%load_ext tensorboard
%tensorboard --logdir=./fairness_indicator_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6008 (pid 72010), started 0:05:58 ago. (Use '!kill 72010' to kill it.)

In [12]:
from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

In [23]:
eval_data = tf.data.TFRecordDataset("/Users/bhuwankarki/Desktop/ops/building_pipelines/ml_pipeline/CsvExampleGen/examples/3/Split-eval/data_tfrecord-00000-of-00001.gz", compression_type="GZIP")

In [25]:
eval_examples = [tf.train.Example.FromString(d.numpy()) for d in eval_data.take(1000)]


In [26]:
eval_examples

[features {
   feature {
     key: "company"
     value {
       bytes_list {
         value: "Stellar Recovery Inc."
       }
     }
   }
   feature {
     key: "company_response"
     value {
       bytes_list {
         value: "Closed with explanation"
       }
     }
   }
   feature {
     key: "consumer_complaint_narrative"
     value {
       bytes_list {
         value: "Almost daily phone calls from Stellar Recovery for months now. All computer calls.   I own my home, own my car, pay my credit card bill monthly, as well as utility bills, insurance, etc.   I have a stellar credit rating, clean credit record, and owe money to no one. Obviously, this is unwarranted harassment.  "
       }
     }
   }
   feature {
     key: "consumer_disputed"
     value {
       int64_list {
         value: 1
       }
     }
   }
   feature {
     key: "issue"
     value {
       bytes_list {
         value: "Cont\'d attempts collect debt not owed"
       }
     }
   }
   feature {
     key: "prod

In [27]:
model = tf.saved_model.load(export_dir="/Users/bhuwankarki/Desktop/ops/building_pipelines/ml_pipeline/Trainer/model/36/Format-Serving")

In [28]:
def predict(examples):
    preds = model.signatures['serving_default'](examples=tf.constant([example.SerializeToString() for example in examples]))
    return preds['outputs'].numpy()

In [29]:
config_builder = WitConfigBuilder(eval_examples).set_custom_predict_fn(predict)

In [31]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
  
!jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
  
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Installing /opt/anaconda3/lib/python3.8/site-packages/tensorflow_model_analysis/static -> tensorflow_model_analysis
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable tensorflow_model_analysis --py --sys-prefix
    
Enabling notebook extension tensorflow_model_analysis/extension...
      - Validating: OK


In [32]:
WitWidget(config_builder)


WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'are_sequence_examples': False, 'inferenc…